# Stable Diffusion Image Variations
A user-friendly notebook for image-to-image generation using a fine-tuned version of Stable Diffusion conditioned on CLIP image embeddings. It uses the code from the [lambad-diffusers](https://github.com/LambdaLabsML/lambda-diffusers) repo.  
GPU acceleration is needed to execute the code in this notebook.

## Settings

Clone the lambda diffusers GitHub repo.

In [ ]:
!git clone https://github.com/LambdaLabsML/lambda-diffusers.git
%cd lambda-diffusers

Install the requirements. Please refer to the code cell below and don't use the provided requirements.txt file, as it doesn't reflect the latest changes.

In [ ]:
!pip install transformers==4.25
!pip install ftfy==6.1.1
!pip instal lPillow==9.2.0
!pip install onnxruntime==1.12.1
!pip installscikit-image==0.19.3
!pip install -e .
!pip install accelerate

Define a function to upload images.

In [ ]:
from google.colab import files

def upload_files():
  uploaded = files.upload()
  for k, v in uploaded.items():
    open(k, 'wb').write(v)
  return list(uploaded.keys())

## Image Generation

Import the dependencies to be used.

In [ ]:
import torchvision.transforms as transforms
from diffusers import StableDiffusionImageVariationPipeline
from PIL import Image

Upload an image to be used as input.

In [ ]:
uploaded_image_list = upload_files()
input_im = Image.open(uploaded_image_list[0])
display(input_im)

Create the pipeline. The code cell below will take few minutes (depending on the network traffic) as it must download the pretrained SD Image Variations model components.

In [ ]:
device = "cuda:0"
sd_pipe = StableDiffusionImageVariationPipeline.from_pretrained(
  "lambdalabs/sd-image-variations-diffusers",
  revision="v2.0",
  )
sd_pipe = sd_pipe.to(device)

Set some Image Variation options.

In [ ]:
guidance_scale = 10 #@param {type:"slider", min:1, max:20, step:0.5}
steps = 50 #@param {type:"slider", min:1, max:100, step:1}
output_filename = "result.jpg" #@param {type: "string"}

Resize, normalize and finally transform the input image into a PyTorch tensor. 

In [ ]:
im = Image.open(uploaded_image_list[0])
tform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(
        (224, 224),
        interpolation=transforms.InterpolationMode.BICUBIC,
        antialias=False,
        ),
    transforms.Normalize(
      [0.48145466, 0.4578275, 0.40821073],
      [0.26862954, 0.26130258, 0.27577711]),
])
inp = tform(im).to(device).unsqueeze(0)

Start image-to-image generation.

In [ ]:
out = sd_pipe(inp, guidance_scale=guidance_scale, num_inference_steps=steps)
out["images"][0].save(output_filename)

Display the generated image.

In [ ]:
output_im = Image.open(output_filename)
display(output_im)